In [14]:
import pandas as pd
import re
from bs4 import BeautifulSoup as BS
import requests

In [20]:


# cookies = {
#     '_ym_uid': '172433126562841683',
#     '_ym_d': '1724331265',
#     '_ym_isad': '2',
#     '_ym_visorc': 'w',
#     '_gid': 'GA1.2.1416790719.1724331265',
#     'XSRF-TOKEN': 'eyJpdiI6IlYyam92ZTZaME5YMUc3aXZWNzl2Z1E9PSIsInZhbHVlIjoieHNsM0V2ZEhqcE5XeUN4UUs0NHV5cU96ZFRGbVlkRmZ3eEJ1K0FuSFJvRXNmWGlZMEtiV3pkTlhOdGw3SmJaUlRYNHhoT3BrT0g3eXBpN0J1QWQ2Rm43b1hsb0VJakJuK2lQQ0hYYWp4eUhIaGhoWWpOTlhkS0RpaFp0S25ITVIiLCJtYWMiOiJjNGMyYmRlZTM2OTc4NzcxMTk0MmIxYjRkM2EzMDA4NzAxMzVjNDRlNGRlNzhmYWRlNmM2NDJmNjI1MmE1ZGIwIiwidGFnIjoiIn0%3D',
#     'pack24ru_session': 'eyJpdiI6IlpFbWxuZFhBUTN2dEVzYjJHWVVrdFE9PSIsInZhbHVlIjoibVZsdEpKNjJpTWVJNEFKaHZGS053N1dvMjU2cHNwRnBwZUladGpLMTFKNjFML0V1QUFYOHFNNXJwQzRsVUZQV0dtQXpwdE16emJVOUY2alpMVi9vZHNUT0p6d2lLTGZkeXY5L2ozbGJtZllMdk5Xa04yWVJIaVpLSFF2Z2hURTciLCJtYWMiOiI1MTA2YzM2N2U5YmI0ZmY2NzFjYTI2OTE4NDQ2MWUxNGRiODgzN2FkMDZhMjNlYWExNWYyMDMzZGUzMTBjY2QzIiwidGFnIjoiIn0%3D',
#     'purchases': 'eyJpdiI6IlY1RllFdnFzSFB3TVZLUHdWVC9XcFE9PSIsInZhbHVlIjoibkUxSEtXalFWaXFRRkRscEFITkRucGxCNWZLdFJ4UHl6cnEzZXVMN1FWaGdGZ1pyZmwvMU51NGVrSDdVSmtVaCIsIm1hYyI6ImVlMWQzNTRiZjBlZTMwNjQ4NGIzNjI0ZDJiMmI5MjVjNmM0M2I5ZjZmNjhhZDI1NzM4YzEwM2Q1N2QzOTMyM2UiLCJ0YWciOiIifQ%3D%3D',
#     '_ga_L53N0EFD2N': 'GS1.1.1724331264.1.1.1724331867.59.0.0',
#     '_ga': 'GA1.2.1132969394.1724331265',
#     '_gat_UA-83288552-1': '1',
# }

# headers = {
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
#     'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,de;q=0.6',
#     'cache-control': 'max-age=0',
#     # 'cookie': '_ym_uid=172433126562841683; _ym_d=1724331265; _ym_isad=2; _ym_visorc=w; _gid=GA1.2.1416790719.1724331265; XSRF-TOKEN=eyJpdiI6IlYyam92ZTZaME5YMUc3aXZWNzl2Z1E9PSIsInZhbHVlIjoieHNsM0V2ZEhqcE5XeUN4UUs0NHV5cU96ZFRGbVlkRmZ3eEJ1K0FuSFJvRXNmWGlZMEtiV3pkTlhOdGw3SmJaUlRYNHhoT3BrT0g3eXBpN0J1QWQ2Rm43b1hsb0VJakJuK2lQQ0hYYWp4eUhIaGhoWWpOTlhkS0RpaFp0S25ITVIiLCJtYWMiOiJjNGMyYmRlZTM2OTc4NzcxMTk0MmIxYjRkM2EzMDA4NzAxMzVjNDRlNGRlNzhmYWRlNmM2NDJmNjI1MmE1ZGIwIiwidGFnIjoiIn0%3D; pack24ru_session=eyJpdiI6IlpFbWxuZFhBUTN2dEVzYjJHWVVrdFE9PSIsInZhbHVlIjoibVZsdEpKNjJpTWVJNEFKaHZGS053N1dvMjU2cHNwRnBwZUladGpLMTFKNjFML0V1QUFYOHFNNXJwQzRsVUZQV0dtQXpwdE16emJVOUY2alpMVi9vZHNUT0p6d2lLTGZkeXY5L2ozbGJtZllMdk5Xa04yWVJIaVpLSFF2Z2hURTciLCJtYWMiOiI1MTA2YzM2N2U5YmI0ZmY2NzFjYTI2OTE4NDQ2MWUxNGRiODgzN2FkMDZhMjNlYWExNWYyMDMzZGUzMTBjY2QzIiwidGFnIjoiIn0%3D; purchases=eyJpdiI6IlY1RllFdnFzSFB3TVZLUHdWVC9XcFE9PSIsInZhbHVlIjoibkUxSEtXalFWaXFRRkRscEFITkRucGxCNWZLdFJ4UHl6cnEzZXVMN1FWaGdGZ1pyZmwvMU51NGVrSDdVSmtVaCIsIm1hYyI6ImVlMWQzNTRiZjBlZTMwNjQ4NGIzNjI0ZDJiMmI5MjVjNmM0M2I5ZjZmNjhhZDI1NzM4YzEwM2Q1N2QzOTMyM2UiLCJ0YWciOiIifQ%3D%3D; _ga_L53N0EFD2N=GS1.1.1724331264.1.1.1724331867.59.0.0; _ga=GA1.2.1132969394.1724331265; _gat_UA-83288552-1=1',
#     'priority': 'u=0, i',
#     'referer': 'https://pack24.ru/kartonnye-korobki',
#     'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
#     'sec-ch-ua-mobile': '?0',
#     'sec-ch-ua-platform': '"Windows"',
#     'sec-fetch-dest': 'document',
#     'sec-fetch-mode': 'navigate',
#     'sec-fetch-site': 'same-origin',
#     'sec-fetch-user': '?1',
#     'upgrade-insecure-requests': '1',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
# }

response = requests.get('https://pack24.ru/kartonnye-korobki/chetyrehklapannye-koroba',
                        # cookies=cookies,
                        # headers=headers
                        )
response

<Response [200]>

In [21]:
soup = BS(response.text, 'html.parser')

In [22]:
t = map(lambda x: str(x), soup.find('nav').next_siblings)
t = '\n'.join(t)

In [23]:
ss = BS(t, 'html.parser')

In [24]:
sku_name = []
sku_href = []
sku_desc = []
price = []
nds = []

item: BS
desc : BS
for item in ss.find_all('div', class_="item__body")[:]:
    sku_name.append(item.find('a', class_='item__name').get_text(strip=True))
    sku_href.append(item.find('a', class_='item__name').get('href'))
    
    if li := item.find('ul', class_='item__desc').find_all('li'):
        item_desc = {}
        for desc in li:
            item_desc[desc.find('span').get_text(strip=True)] = re.sub(r'\n*\s+', ' ', desc.find('span').next_sibling.get_text(strip=True))
        sku_desc.append(item_desc)
    
    price.append(re.findall(r'(\d+.\d+)', item.find('div', class_='item__price').find('span').previous_element.get_text(strip=True))[0])
    try:
        nds.append(re.findall(r'(\d+.\d+)', item.find('div', class_='item__price').find('div', class_='item__tooltip').find('p').find_next_sibling('p').get_text(strip=True))[0])
    except AttributeError:
        nds.append(None)
    # print(item.find('div', class_='item__price').find('div').find('div').find('p').find_next_sibling('p').get_text(strip=True))
    
    

In [25]:
pd.DataFrame({
    'sku_name': sku_name,
    'sku_href': sku_href,
    'sku_desc': sku_desc,
    'price': price,
    'nds': nds,
})

,sku_name,sku_href,sku_desc,price,nds
0,Гофрокороб 80*80*80 мм,https://pack24.ru/kartonnye-korobki/chetyrehkl...,"{'Размер': '80*80*80 мм (Д*Ш*В), объем 0,512 л...",10.55,1.76
1,Гофрокороб 100*100*100 мм,https://pack24.ru/kartonnye-korobki/chetyrehkl...,"{'Размер': '100*100*100 мм (Д*Ш*В), объем - 1 ...",10.75,1.79
2,Гофрокороб 100*100*200 мм,https://pack24.ru/kartonnye-korobki/chetyrehkl...,"{'Размер': '100*100*200 мм (Д*Ш*В), объем - 2 ...",15.70,2.62
3,Гофрокороб 120*120*120 мм,https://pack24.ru/kartonnye-korobki/chetyrehkl...,"{'Размер': '120*120*120 мм (Д*Ш*В), объем 1,73...",10.95,1.83
4,Гофрокороб 135*135*135 мм,https://pack24.ru/kartonnye-korobki/chetyrehkl...,"{'Размер': '135*135*135 мм (Д*Ш*В), объем 2,46...",11.95,1.99
...,...,...,...,...,...
60,"Архивный короб А4, 330*230*230 мм, с крышкой и...",https://pack24.ru/kartonnye-korobki/arhivnye-k...,"{'Размер': '330*230*230 мм(Д*Ш*В), полезный об...",89.00,14.83
61,"Почтовая коробка Тип Е, №1, (265*165*50), без ...",https://pack24.ru/kartonnye-korobki/pochtovye-...,"{'Размер': '265*165*50 мм (Д*Ш*В)', 'Материал'...",21.90,3.65
62,"Почтовая коробка Тип Ж, (175*120*100), без лог...",https://pack24.ru/kartonnye-korobki/pochtovye-...,"{'Размер': '175*120*100 мм (Д*Ш*В)', 'Материал...",19.50,3.25
63,"Стрейч-пленка 2 кг, вторичное сырье",https://pack24.ru/strejch-plyonka/strejch-plen...,"{'Размер': 'Ширина 50 см, намотка 220 м', 'Мат...",575.00,95.83
